In [1]:
import sys
import os
import time
import bilby
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np

from few.trajectory.inspiral import EMRIInspiral
from few.amplitude.romannet import RomanAmplitude
from few.amplitude.interp2dcubicspline import Interp2DAmplitude
from few.waveform import FastSchwarzschildEccentricFlux, SlowSchwarzschildEccentricFlux, GenerateEMRIWaveform
from few.utils.utility import (get_overlap, 
                               get_mismatch, 
                               get_fundamental_frequencies, 
                               get_separatrix, 
                               get_mu_at_t, 
                               get_p_at_t, 
                               get_kerr_geo_constants_of_motion,
                               xI_to_Y,
                               Y_to_xI)

from few.utils.ylm import GetYlms
from few.utils.modeselector import ModeSelector
from few.summation.interpolatedmodesum import CubicSplineInterpolant
from few.waveform import SchwarzschildEccentricWaveformBase
from few.summation.interpolatedmodesum import InterpolatedModeSum
from few.summation.directmodesum import DirectModeSum
from few.utils.constants import *
from few.summation.aakwave import AAKSummation
from few.waveform import Pn5AAKWaveform, AAKWaveformBase
from few.utils.baseclasses import Pn5AAK, ParallelModuleBase
from scipy import optimize
import scipy


use_gpu = False

# keyword arguments for inspiral generator (RunSchwarzEccFluxInspiral)
inspiral_kwargs={
        "DENSE_STEPPING": 0,  # we want a sparsely sampled trajectory
        "max_init_len": int(1e3),  # all of the trajectories will be well under len = 1000
    }

# keyword arguments for inspiral generator (RomanAmplitude)
amplitude_kwargs = {
    "max_init_len": int(1e3),  # all of the trajectories will be well under len = 1000
    "use_gpu": use_gpu  # GPU is available in this class
}

# keyword arguments for Ylm generator (GetYlms)
Ylm_kwargs = {
    "assume_positive_m": False  # if we assume positive m, it will generate negative m for all m>0
}

# keyword arguments for summation generator (InterpolatedModeSum)
sum_kwargs = {
    "use_gpu": use_gpu,  # GPU is availabel for this type of summation
    "pad_output": False,
}
traj=EMRIInspiral(func="SchwarzEccFluxwithscalar")

In [2]:
# parameters
T =1.0  # years
dt = 10.0  # seconds
M = 1e6
a0 = 0.0  # will be ignored in Schwarzschild waveform
mu = 1e1
p0 = 9.46407266656996
e0 = 0.1
Y0 = 1.0  # will be ignored in Schwarzschild waveform

qK = np.pi/4  # polar spin angle
phiK = np.pi/4  # azimuthal viewing angle
qS = np.pi/3 # polar sky angle
phiS = np.pi/2  # azimuthal viewing angle
dist = 0.1 # distance

Phi_phi0 = 0.0
Phi_theta0 = 0.0
Phi_r0 = 0.0

d=0.05


traj_args = [M, mu, 0.0, e0, 1.0,d]
traj_kwargs = {}
index_of_p = 3

t_out = 1.0

p_new = get_p_at_t(
    traj,
    t_out,
    traj_args,
    index_of_p=3,
    index_of_a=2,
    index_of_e=4,
    index_of_x=5,
    traj_kwargs={},
    xtol=2e-12,
    rtol=8.881784197001252e-16,
    bounds=None,
)
print(p_new)
t, p, e, x, Phi_phi, Phi_theta, Phi_r = traj(M, mu, 
                                             0.0, p_new, e0, 1.0,d,T=T)#,upsample=True,dt=dt,fix_t=True)
new_t=np.array([])
for i in range(1,len(t)):
    if i==len(t)-1:
        mid=np.linspace(t[i-1],t[i],10,endpoint=True)
    else:
        mid=np.linspace(t[i-1],t[i],10,endpoint=False)
    new_t=np.append(new_t,mid)
print(new_t[-1]/YRSID_SI)





p11=[-1,25/2,-75,300,-1050,0,1050,-300,75,-25/2,1]

partialresult1=[]

injection_parameters=np.array([M,mu,p_new,e0,d])
bilby.core.utils.check_directory_exists_and_if_not_mkdir('orbit{}'.format(e0))

filenames=['orbit{}/pm.txt'.format(e0),'orbit{}/pmu.txt'.format(e0),
           'orbit{}/pp.txt'.format(e0),'orbit{}/pe.txt'.format(e0),'orbit{}/pd.txt'.format(e0)]

delta_paramters=[1e-5,1e-5,1e-6,1e-3,1e-2]

for i in [0,1,2,3,4]:
    dp=0
    de=0
    dPhi_phi=0
    dPhi_r=0
    domegaphi=0
    domegar=0
    lenthdp=0

    for k,j in enumerate(np.arange(11)-5):

        zeros=np.zeros_like(injection_parameters)

        zeros[i]=1

        para=injection_parameters*(1+delta_paramters[i]*zeros*j)

        deltatheta=injection_parameters[i]*delta_paramters[i]

        t, p, e, x, Phi_phi, Phi_theta, Phi_r = traj(para[0], para[1], 
                                                     0.0, para[2], para[3], 1.0 ,para[4],T=T,new_t=new_t,upsample=True,fix_t=True)
        lentht=len(t)
        
        if isinstance(dp,np.ndarray):
            if lentht<=lenthdp:
                dp=dp[:lentht]
                de=de[:lentht]
                dPhi_phi=dPhi_phi[:lentht]
                dPhi_r=dPhi_r[:lentht]
                domegaphi=domegaphi[:lentht]
                domegar=domegar[:lentht]
            else:
                t=t[:lenthdp]
                p=p[:lenthdp]
                e=e[:lenthdp]
                Phi_phi=Phi_phi[:lenthdp]
                Phi_r=Phi_r[:lenthdp]

        omega=np.array(get_fundamental_frequencies(0.0,p,e,1.0))
        dp=dp+p*p11[k]/(1260*deltatheta)
        de=de+e*p11[k]/(1260*deltatheta)  
        dPhi_phi=dPhi_phi+Phi_phi*p11[k]/(1260*deltatheta)  
        dPhi_r=dPhi_r+Phi_r*p11[k]/(1260*deltatheta)  
        domegaphi=domegaphi+omega[0,:]*p11[k]/(1260*deltatheta)
        domegar=domegar+omega[2,:]*p11[k]/(1260*deltatheta)  
        lenthdp=len(dp)
        
    tarray=new_t[:lenthdp]
    np.savetxt(filenames[i],np.array([tarray,dp,de,dPhi_phi,dPhi_r,domegaphi,domegar]).T)
    
tlist, plist, elist, xlist, Phi_philist, Phi_thetalist, Phi_rlist = traj(injection_parameters[0], injection_parameters[1], 0.0, 
                                             injection_parameters[2], injection_parameters[3], 1.0 ,
                                             injection_parameters[4],T=T,new_t=new_t,upsample=True,fix_t=True)  
omega=np.array(get_fundamental_frequencies(0.0,plist,elist,1.0))
omegaphi=omega[0,:]
omegar=omega[2,:]
np.savetxt('orbit{}/orbit.txt'.format(e0),np.array([tlist,plist,elist,Phi_philist,Phi_rlist,omegaphi,omegar]).T)
#     partialresult1.append([tarray,dp,de,dPhi_phi,dPhi_r,domegaphi,domegar])
# parameters=['dp','de','dPhi_phi','dPhi_r','domegaphi','domegar']

9.477218521085783
0.9999999546137555


In [2]:
# parameters
T =1.0  # years
dt = 10.0  # seconds
M = 1e6
a0 = 0.0  # will be ignored in Schwarzschild waveform
mu = 1e1
p0 = 9.46407266656996
e0 = 0.29
Y0 = 1.0  # will be ignored in Schwarzschild waveform

qK = np.pi/4  # polar spin angle
phiK = np.pi/4  # azimuthal viewing angle
qS = np.pi/3 # polar sky angle
phiS = np.pi/2  # azimuthal viewing angle
dist = 0.1 # distance

Phi_phi0 = 0.0
Phi_theta0 = 0.0
Phi_r0 = 0.0

d=0.05


traj_args = [M, mu, 0.0, e0, 1.0,d]
traj_kwargs = {}
index_of_p = 3

t_out = 1.0

p_new = get_p_at_t(
    traj,
    t_out,
    traj_args,
    index_of_p=3,
    index_of_a=2,
    index_of_e=4,
    index_of_x=5,
    traj_kwargs={},
    xtol=2e-12,
    rtol=8.881784197001252e-16,
    bounds=None,
)
print(p_new)
t, p, e, x, Phi_phi, Phi_theta, Phi_r = traj(M, mu, 
                                             0.0, p_new, e0, 1.0,d,T=T)#,upsample=True,dt=dt,fix_t=True)
new_t=np.array([])
for i in range(1,len(t)):
    if i==len(t)-1:
        mid=np.linspace(t[i-1],t[i],10,endpoint=True)
    else:
        mid=np.linspace(t[i-1],t[i],10,endpoint=False)
    new_t=np.append(new_t,mid)
print(new_t[-1]/YRSID_SI)





p11=[-1,25/2,-75,300,-1050,0,1050,-300,75,-25/2,1]

partialresult1=[]

injection_parameters=np.array([M,mu,p_new,e0,d])
bilby.core.utils.check_directory_exists_and_if_not_mkdir('orbit{}'.format(e0))

filenames=['orbit{}/pm.txt'.format(e0),'orbit{}/pmu.txt'.format(e0),
           'orbit{}/pp.txt'.format(e0),'orbit{}/pe.txt'.format(e0),'orbit{}/pd.txt'.format(e0)]

delta_paramters=[1e-5,1e-5,1e-6,1e-3,1e-2]

for i in [0,1,2,3,4]:
    dp=0
    de=0
    dPhi_phi=0
    dPhi_r=0
    domegaphi=0
    domegar=0
    lenthdp=0

    for k,j in enumerate(np.arange(11)-5):

        zeros=np.zeros_like(injection_parameters)

        zeros[i]=1

        para=injection_parameters*(1+delta_paramters[i]*zeros*j)

        deltatheta=injection_parameters[i]*delta_paramters[i]

        t, p, e, x, Phi_phi, Phi_theta, Phi_r = traj(para[0], para[1], 
                                                     0.0, para[2], para[3], 1.0 ,para[4],T=T,new_t=new_t,upsample=True,fix_t=True)
        lentht=len(t)
        
        if isinstance(dp,np.ndarray):
            if lentht<=lenthdp:
                dp=dp[:lentht]
                de=de[:lentht]
                dPhi_phi=dPhi_phi[:lentht]
                dPhi_r=dPhi_r[:lentht]
                domegaphi=domegaphi[:lentht]
                domegar=domegar[:lentht]
            else:
                t=t[:lenthdp]
                p=p[:lenthdp]
                e=e[:lenthdp]
                Phi_phi=Phi_phi[:lenthdp]
                Phi_r=Phi_r[:lenthdp]

        omega=np.array(get_fundamental_frequencies(0.0,p,e,1.0))
        dp=dp+p*p11[k]/(1260*deltatheta)
        de=de+e*p11[k]/(1260*deltatheta)  
        dPhi_phi=dPhi_phi+Phi_phi*p11[k]/(1260*deltatheta)  
        dPhi_r=dPhi_r+Phi_r*p11[k]/(1260*deltatheta)  
        domegaphi=domegaphi+omega[0,:]*p11[k]/(1260*deltatheta)
        domegar=domegar+omega[2,:]*p11[k]/(1260*deltatheta)  
        lenthdp=len(dp)
        
    tarray=new_t[:lenthdp]
    np.savetxt(filenames[i],np.array([tarray,dp,de,dPhi_phi,dPhi_r,domegaphi,domegar]).T)
    
tlist, plist, elist, xlist, Phi_philist, Phi_thetalist, Phi_rlist = traj(injection_parameters[0], injection_parameters[1], 0.0, 
                                             injection_parameters[2], injection_parameters[3], 1.0 ,
                                             injection_parameters[4],T=T,new_t=new_t,upsample=True,fix_t=True)  
omega=np.array(get_fundamental_frequencies(0.0,plist,elist,1.0))
omegaphi=omega[0,:]
omegar=omega[2,:]
np.savetxt('orbit{}/orbit.txt'.format(e0),np.array([tlist,plist,elist,Phi_philist,Phi_rlist,omegaphi,omegar]).T)
#     partialresult1.append([tarray,dp,de,dPhi_phi,dPhi_r,domegaphi,domegar])
# parameters=['dp','de','dPhi_phi','dPhi_r','domegaphi','domegar']

9.526631180577251
0.9999995795167056


In [3]:
# parameters
T =1.0  # years
dt = 10.0  # seconds
M = 1e6
a0 = 0.0  # will be ignored in Schwarzschild waveform
mu = 1e1
p0 = 9.46407266656996
e0 = 0.25
Y0 = 1.0  # will be ignored in Schwarzschild waveform

qK = np.pi/4  # polar spin angle
phiK = np.pi/4  # azimuthal viewing angle
qS = np.pi/3 # polar sky angle
phiS = np.pi/2  # azimuthal viewing angle
dist = 0.1 # distance

Phi_phi0 = 0.0
Phi_theta0 = 0.0
Phi_r0 = 0.0

d=0.05


traj_args = [M, mu, 0.0, e0, 1.0,d]
traj_kwargs = {}
index_of_p = 3

t_out = 1.0

p_new = get_p_at_t(
    traj,
    t_out,
    traj_args,
    index_of_p=3,
    index_of_a=2,
    index_of_e=4,
    index_of_x=5,
    traj_kwargs={},
    xtol=2e-12,
    rtol=8.881784197001252e-16,
    bounds=None,
)
print(p_new)
t, p, e, x, Phi_phi, Phi_theta, Phi_r = traj(M, mu, 
                                             0.0, p_new, e0, 1.0,d,T=T)#,upsample=True,dt=dt,fix_t=True)
new_t=np.array([])
for i in range(1,len(t)):
    if i==len(t)-1:
        mid=np.linspace(t[i-1],t[i],10,endpoint=True)
    else:
        mid=np.linspace(t[i-1],t[i],10,endpoint=False)
    new_t=np.append(new_t,mid)
print(new_t[-1]/YRSID_SI)





p11=[-1,25/2,-75,300,-1050,0,1050,-300,75,-25/2,1]

partialresult1=[]

injection_parameters=np.array([M,mu,p_new,e0,d])
bilby.core.utils.check_directory_exists_and_if_not_mkdir('orbit{}'.format(e0))

filenames=['orbit{}/pm.txt'.format(e0),'orbit{}/pmu.txt'.format(e0),
           'orbit{}/pp.txt'.format(e0),'orbit{}/pe.txt'.format(e0),'orbit{}/pd.txt'.format(e0)]

delta_paramters=[1e-5,1e-5,1e-6,1e-3,1e-2]

for i in [0,1,2,3,4]:
    dp=0
    de=0
    dPhi_phi=0
    dPhi_r=0
    domegaphi=0
    domegar=0
    lenthdp=0

    for k,j in enumerate(np.arange(11)-5):

        zeros=np.zeros_like(injection_parameters)

        zeros[i]=1

        para=injection_parameters*(1+delta_paramters[i]*zeros*j)

        deltatheta=injection_parameters[i]*delta_paramters[i]

        t, p, e, x, Phi_phi, Phi_theta, Phi_r = traj(para[0], para[1], 
                                                     0.0, para[2], para[3], 1.0 ,para[4],T=T,new_t=new_t,upsample=True,fix_t=True)
        lentht=len(t)
        
        if isinstance(dp,np.ndarray):
            if lentht<=lenthdp:
                dp=dp[:lentht]
                de=de[:lentht]
                dPhi_phi=dPhi_phi[:lentht]
                dPhi_r=dPhi_r[:lentht]
                domegaphi=domegaphi[:lentht]
                domegar=domegar[:lentht]
            else:
                t=t[:lenthdp]
                p=p[:lenthdp]
                e=e[:lenthdp]
                Phi_phi=Phi_phi[:lenthdp]
                Phi_r=Phi_r[:lenthdp]

        omega=np.array(get_fundamental_frequencies(0.0,p,e,1.0))
        dp=dp+p*p11[k]/(1260*deltatheta)
        de=de+e*p11[k]/(1260*deltatheta)  
        dPhi_phi=dPhi_phi+Phi_phi*p11[k]/(1260*deltatheta)  
        dPhi_r=dPhi_r+Phi_r*p11[k]/(1260*deltatheta)  
        domegaphi=domegaphi+omega[0,:]*p11[k]/(1260*deltatheta)
        domegar=domegar+omega[2,:]*p11[k]/(1260*deltatheta)  
        lenthdp=len(dp)
        
    tarray=new_t[:lenthdp]
    np.savetxt(filenames[i],np.array([tarray,dp,de,dPhi_phi,dPhi_r,domegaphi,domegar]).T)
    
tlist, plist, elist, xlist, Phi_philist, Phi_thetalist, Phi_rlist = traj(injection_parameters[0], injection_parameters[1], 0.0, 
                                             injection_parameters[2], injection_parameters[3], 1.0 ,
                                             injection_parameters[4],T=T,new_t=new_t,upsample=True,fix_t=True)  
omega=np.array(get_fundamental_frequencies(0.0,plist,elist,1.0))
omegaphi=omega[0,:]
omegar=omega[2,:]
np.savetxt('orbit{}/orbit.txt'.format(e0),np.array([tlist,plist,elist,Phi_philist,Phi_rlist,omegaphi,omegar]).T)
#     partialresult1.append([tarray,dp,de,dPhi_phi,dPhi_r,domegaphi,domegar])
# parameters=['dp','de','dPhi_phi','dPhi_r','domegaphi','domegar']

9.51338236601256
0.9999999819849438


In [3]:
# parameters
T =1.0  # years
dt = 10.0  # seconds
M = 1e6
a0 = 0.0  # will be ignored in Schwarzschild waveform
mu = 1e1
p0 = 9.46407266656996
e0 = 0.01
Y0 = 1.0  # will be ignored in Schwarzschild waveform

qK = np.pi/4  # polar spin angle
phiK = np.pi/4  # azimuthal viewing angle
qS = np.pi/3 # polar sky angle
phiS = np.pi/2  # azimuthal viewing angle
# dist = 0.1 # distance

Phi_phi0 = 0.0
Phi_theta0 = 0.0
Phi_r0 = 0.0

d=0.05


traj_args = [M, mu, 0.0, e0, 1.0,d]
traj_kwargs = {}
index_of_p = 3

t_out = 1.0

p_new = get_p_at_t(
    traj,
    t_out,
    traj_args,
    index_of_p=3,
    index_of_a=2,
    index_of_e=4,
    index_of_x=5,
    traj_kwargs={},
    xtol=2e-12,
    rtol=8.881784197001252e-16,
    bounds=None,
)
print(p_new)
t, p, e, x, Phi_phi, Phi_theta, Phi_r = traj(M, mu, 
                                             0.0, p_new, e0, 1.0,d,T=T)#,upsample=True,dt=dt,fix_t=True)
new_t=np.array([])
for i in range(1,len(t)):
    if i==len(t)-1:
        mid=np.linspace(t[i-1],t[i],10,endpoint=True)
    else:
        mid=np.linspace(t[i-1],t[i],10,endpoint=False)
    new_t=np.append(new_t,mid)
print(new_t[-1]/YRSID_SI)





p11=[-1,25/2,-75,300,-1050,0,1050,-300,75,-25/2,1]

partialresult1=[]

injection_parameters=np.array([M,mu,p_new,e0,d])
bilby.core.utils.check_directory_exists_and_if_not_mkdir('orbit{}'.format(e0))

filenames=['orbit{}/pm.txt'.format(e0),'orbit{}/pmu.txt'.format(e0),
           'orbit{}/pp.txt'.format(e0),'orbit{}/pe.txt'.format(e0),'orbit{}/pd.txt'.format(e0)]

delta_paramters=[1e-5,1e-5,1e-6,5e-2,1e-2]

for i in [0,1,2,3,4]:
    dp=0
    de=0
    dPhi_phi=0
    dPhi_r=0
    domegaphi=0
    domegar=0
    lenthdp=0

    for k,j in enumerate(np.arange(11)-5):

        zeros=np.zeros_like(injection_parameters)

        zeros[i]=1

        para=injection_parameters*(1+delta_paramters[i]*zeros*j)

        deltatheta=injection_parameters[i]*delta_paramters[i]

        t, p, e, x, Phi_phi, Phi_theta, Phi_r = traj(para[0], para[1], 
                                                     0.0, para[2], para[3], 1.0 ,para[4],T=T,new_t=new_t,upsample=True,fix_t=True)
        lentht=len(t)
        
        if isinstance(dp,np.ndarray):
            if lentht<=lenthdp:
                dp=dp[:lentht]
                de=de[:lentht]
                dPhi_phi=dPhi_phi[:lentht]
                dPhi_r=dPhi_r[:lentht]
                domegaphi=domegaphi[:lentht]
                domegar=domegar[:lentht]
            else:
                t=t[:lenthdp]
                p=p[:lenthdp]
                e=e[:lenthdp]
                Phi_phi=Phi_phi[:lenthdp]
                Phi_r=Phi_r[:lenthdp]

        omega=np.array(get_fundamental_frequencies(0.0,p,e,1.0))
        dp=dp+p*p11[k]/(1260*deltatheta)
        de=de+e*p11[k]/(1260*deltatheta)  
        dPhi_phi=dPhi_phi+Phi_phi*p11[k]/(1260*deltatheta)  
        dPhi_r=dPhi_r+Phi_r*p11[k]/(1260*deltatheta)  
        domegaphi=domegaphi+omega[0,:]*p11[k]/(1260*deltatheta)
        domegar=domegar+omega[2,:]*p11[k]/(1260*deltatheta)  
        lenthdp=len(dp)
        
    tarray=new_t[:lenthdp]
    np.savetxt(filenames[i],np.array([tarray,dp,de,dPhi_phi,dPhi_r,domegaphi,domegar]).T)
    
tlist, plist, elist, xlist, Phi_philist, Phi_thetalist, Phi_rlist = traj(injection_parameters[0], injection_parameters[1], 0.0, 
                                             injection_parameters[2], injection_parameters[3], 1.0 ,
                                             injection_parameters[4],T=T,new_t=new_t,upsample=True,fix_t=True)  
omega=np.array(get_fundamental_frequencies(0.0,plist,elist,1.0))
omegaphi=omega[0,:]
omegar=omega[2,:]
np.savetxt('orbit{}/orbit.txt'.format(e0),np.array([tlist,plist,elist,Phi_philist,Phi_rlist,omegaphi,omegar]).T)
#     partialresult1.append([tarray,dp,de,dPhi_phi,dPhi_r,domegaphi,domegar])
# parameters=['dp','de','dPhi_phi','dPhi_r','domegaphi','domegar']

9.466769760512216
1.0000000182139626


In [9]:
# parameters
T =1.0  # years
dt = 10.0  # seconds
M = 1e6
a0 = 0.0  # will be ignored in Schwarzschild waveform
mu = 1e1
p0 = 9.46407266656996
e0 = 0.2
Y0 = 1.0  # will be ignored in Schwarzschild waveform

qK = np.pi/4  # polar spin angle
phiK = np.pi/4  # azimuthal viewing angle
qS = np.pi/3 # polar sky angle
phiS = np.pi/2  # azimuthal viewing angle
dist = 0.1 # distance

Phi_phi0 = 0.0
Phi_theta0 = 0.0
Phi_r0 = 0.0

d=0.05


traj_args = [M, mu, 0.0, e0, 1.0,d]
traj_kwargs = {}
index_of_p = 3

t_out = 1.0

p_new = get_p_at_t(
    traj,
    t_out,
    traj_args,
    index_of_p=3,
    index_of_a=2,
    index_of_e=4,
    index_of_x=5,
    traj_kwargs={},
    xtol=2e-12,
    rtol=8.881784197001252e-16,
    bounds=None,
)
print(p_new)
t, p, e, x, Phi_phi, Phi_theta, Phi_r = traj(M, mu, 
                                             0.0, p_new, e0, 1.0,d,T=T)#,upsample=True,dt=dt,fix_t=True)
new_t=np.array([])
for i in range(1,len(t)):
    if i==len(t)-1:
        mid=np.linspace(t[i-1],t[i],10,endpoint=True)
    else:
        mid=np.linspace(t[i-1],t[i],10,endpoint=False)
    new_t=np.append(new_t,mid)
print(new_t[-1]/YRSID_SI)


p11=[-1,25/2,-75,300,-1050,0,1050,-300,75,-25/2,1]

partialresult1=[]

injection_parameters=np.array([M,mu,p_new,e0,d])
bilby.core.utils.check_directory_exists_and_if_not_mkdir('orbitd{}'.format(d))

filenames=['orbitd{}/pm.txt'.format(d),'orbitd{}/pmu.txt'.format(d),
           'orbitd{}/pp.txt'.format(d),'orbitd{}/pe.txt'.format(d),'orbitd{}/pd.txt'.format(d)]

delta_paramters=[1e-5,1e-5,1e-6,1e-3,0.1]

for i in [0,1,2,3,4]:
    dp=0
    de=0
    dPhi_phi=0
    dPhi_r=0
    domegaphi=0
    domegar=0
    lenthdp=0

    for k,j in enumerate(np.arange(11)-5):

        zeros=np.zeros_like(injection_parameters)

        zeros[i]=1

        para=injection_parameters*(1+delta_paramters[i]*zeros*j)

        deltatheta=injection_parameters[i]*delta_paramters[i]

        t, p, e, x, Phi_phi, Phi_theta, Phi_r = traj(para[0], para[1], 
                                                     0.0, para[2], para[3], 1.0 ,para[4],T=T,new_t=new_t,upsample=True,fix_t=True)
        lentht=len(t)
        
        if isinstance(dp,np.ndarray):
            if lentht<=lenthdp:
                dp=dp[:lentht]
                de=de[:lentht]
                dPhi_phi=dPhi_phi[:lentht]
                dPhi_r=dPhi_r[:lentht]
                domegaphi=domegaphi[:lentht]
                domegar=domegar[:lentht]
            else:
                t=t[:lenthdp]
                p=p[:lenthdp]
                e=e[:lenthdp]
                Phi_phi=Phi_phi[:lenthdp]
                Phi_r=Phi_r[:lenthdp]

        omega=np.array(get_fundamental_frequencies(0.0,p,e,1.0))
        dp=dp+p*p11[k]/(1260*deltatheta)
        de=de+e*p11[k]/(1260*deltatheta)  
        dPhi_phi=dPhi_phi+Phi_phi*p11[k]/(1260*deltatheta)  
        dPhi_r=dPhi_r+Phi_r*p11[k]/(1260*deltatheta)  
        domegaphi=domegaphi+omega[0,:]*p11[k]/(1260*deltatheta)
        domegar=domegar+omega[2,:]*p11[k]/(1260*deltatheta)  
        lenthdp=len(dp)
        
    tarray=new_t[:lenthdp]
    np.savetxt(filenames[i],np.array([tarray,dp,de,dPhi_phi,dPhi_r,domegaphi,domegar]).T)
    
tlist, plist, elist, xlist, Phi_philist, Phi_thetalist, Phi_rlist = traj(injection_parameters[0], injection_parameters[1], 0.0, 
                                             injection_parameters[2], injection_parameters[3], 1.0 ,
                                             injection_parameters[4],T=T,new_t=new_t,upsample=True,fix_t=True)  
omega=np.array(get_fundamental_frequencies(0.0,plist,elist,1.0))
omegaphi=omega[0,:]
omegar=omega[2,:]
np.savetxt('orbitd{}/orbit.txt'.format(d),np.array([tlist,plist,elist,Phi_philist,Phi_rlist,omegaphi,omegar]).T)
#     partialresult1.append([tarray,dp,de,dPhi_phi,dPhi_r,domegaphi,domegar])
# parameters=['dp','de','dPhi_phi','dPhi_r','domegaphi','domegar']

9.499004986290165
0.999999779035765
